In [62]:
from IPython.display import display, Markdown, HTML
from pprint import pformat
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
!pip install requests lxml

In [86]:
import requests
import re
import lxml.etree as etree

**Commit ID zum Zeitpunkt des Forks**: 0abd9426cecc335a86a30a8bc9c73d685f204458

# 1. nlp/data API liefert keine eingeschachtelten NEs (hier LOCs in ORGS)

## document API

Nachfolgend z.B. `<orgName n="f3f28d9d-fb72-4e8d-b28d-9dfb72be8d61">Arbeitserziehungsanstalt <placeName n="ce3ebcfc-d66f-4acd-bebc-fcd66fbacddc">Uitikon</placeName></orgName>`

In [93]:
r = requests.get('http://localhost:8080/exist/apps/tei-publisher/api/document/annotate/MM_24_101_KRP_1980_042_0001.xml')
r_content = r.content.decode('utf-8')
print(etree.tostring(etree.fromstring(r_content), pretty_print=True, encoding='unicode'))

<TEI xmlns="http://www.tei-c.org/ns/1.0">
  <teiHeader>
    <fileDesc>
      <titleStmt>
        <title>Verhandlungsgegenstände; Geschäftsordnung</title>
        <respStmt>
          <resp ref="#OCR/dmr" key="transcript">Transkript: OCR/dmr<date when="2011-11-07">07.11.2011</date></resp>
          <name key="editor">Staatsarchiv des Kantons Zürich</name>
        </respStmt>
      </titleStmt>
      <publicationStmt>
        <authority>Staatsarchiv des Kantons Zürich</authority>
        <date>2011</date>
        <pubPlace>Zürich</pubPlace>
      </publicationStmt>
      <seriesStmt>
        <title>Kantonsratsprotokolle seit 1803 online</title>
        <editor>Staatsarchiv des Kantons Zürich</editor>
      </seriesStmt>
      <sourceDesc>
        <bibl>
          <title>Verhandlungsgegenstände; Geschäftsordnung</title>
          <date when="1980-02-04">04.02.1980</date>
          <ident>StAZH MM 24.101 KRP 1980/042/0001</ident>
          <idno>2437</idno>
          <edition>Kantonsratspr

## nlp/data API

In [56]:
from IPython.display import Markdown

r = requests.get('http://localhost:8080/exist/apps/tei-publisher/api/nlp/data/annotate/MM_24_101_KRP_1980_042_0001.xml')
for p in r.json():
    p_str = pformat(p)
    if 'Arbeitserziehungsanstalt Uitikon' in p_str:
        display(Markdown(f'<pre><b>{p_str}</b></pre>'))
    else:
        display(Markdown(f'```\n{p_str}`\n```'))

```
{'entities': [],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': ''}`
```

```
{'entities': [],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': '1. Mitteilungen'}`
```

```
{'entities': [[49, 86, 'ORG'], [112, 127, 'PER'], [129, 139, 'LOC']],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': '2. Wahl eines Mitglieds des Verwaltungsrates der Elektrizitätswerke '
         'des Kantons Zürich für den zurückgetretenen Jakob Bachofner, '
         'Fehraltorf'}`
```

<pre><b>{'entities': [[26, 60, 'ORG'], [69, 101, 'ORG']],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': '3. Jahresbericht 1978 der kantonalen Strafanstalt Regensdorf und der '
         'Arbeitserziehungsanstalt Uitikon (Antrag der '
         'Geschäftsprüfungskommission vom 11. Januar 1980)'}</b></pre>

```
{'entities': [[34, 54, 'ORG'], [195, 207, 'ORG']],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': '4. Erhöhung des Grundkapitals der Zürcher Kantonalbank (Antrag des '
         'Bankrates vom 25. Oktober 1979 und gleichlautender Antrag der '
         'Kommission zur Prüfung der Rechnung und des Geschäftsberichts der '
         'Kantonalbank vom 19. Dezember 1979) 2354'}`
```

```
{'entities': [[111, 122, 'ORGderiv'], [148, 163, 'ORG']],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': '5. Änderung der Kirchengesetzgebung a) Gesetz über die '
         'evangelisch-reformierte Landeskirche b) Gesetz über das katholische '
         'Kirchenwesen (Antrag des Regierungsrates vom 22. August 1979 und '
         'abgeänderter Antrag der Kommission vom 15. Januar 1980) 2330a'}`
```

```
{'entities': [[17, 29, 'ORG'], [115, 150, 'ORG'], [163, 178, 'ORG']],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': '6. Beschluss des Kantonsrates über die Erhöhung des Staatsbeitrages '
         'an die Abteilung Gestalterische Lehrberufe der Kunstgewerbeschule '
         'der Stadt Zürich (Antrag des Regierungsrates vom 10. Oktober 1979 '
         'und abgeänderter Antrag der Kommission vom 10. Januar 1980) 2334a'}`
```

```
{'entities': [],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': 'Geschäftsordnung'}`
```

```
{'entities': [],
 'source': 'annotate/MM_24_101_KRP_1980_042_0001.xml',
 'text': 'Das Wort zur Traktandenliste wird nicht verlangt; sie ist genehmigt.'}`
```

# 2. nlp/text API liefert Inhalt von Tabellenzellen leerzeichengetrennt

Dabei kann es sich auch um einzelne Leerzeichen handeln, was eine Unterscheidung von normalen Leerzeichen verunmöglicht.

Überlegung: Falls stattdessem Tabulatoren geliefert würden, liessen sich diese leicht durch Kommas substituieren, was die NER erheblich erleichtert.

  <table>
    <tr>
        <th>Gemeinde:</th>
        <th>Name des Rekurrenten:</th>
        <th>Geschäftszweig:</th>
        <th>Taxation pro 1863, 1864</th>
        <th>Beschluss.</th>
    </tr>
    <tr>
        <th>
            <hi rend="underline">I. Section:</hi>
        </th>
    </tr>
    <tr>
        <th/>
        <th/>
        <th/>
        <th>   <hi rend="underline">Frk</hi>
        </th>
        <th>     <hi rend="underline">Frk.</hi>
        </th>
    </tr>
    <tr>
        <th>Hirslanden</th>
        <th>Hottinger, Konrad</th>
        <th>Seidenfabrikant</th>
        <th>30</th>
        <th>30</th>
    </tr>
    <tr>
        <th>Wollishofen</th>
        <th>Asper L.</th>
        <th>Seidenstofffabrik</th>
        <th>40</th>
        <th>40</th>
    </tr>
    <tr>
        <th>Zürich</th>
        <th>Andreae, Otto,</th>
        <th>Commission in Seidenwaaren</th>
        <th>300</th>
        <th>300.</th>
    </tr>
    <tr>
        <th>    “</th>
        <th>Bäbler, Gebrüder</th>
        <th>Commiss. Verkauf v. Rohstoff</th>
        <th>100</th>
        <th>100.</th>
    </tr>
</table>

In [67]:
r = requests.get('http://localhost:8080/exist/apps/tei-publisher/api/nlp/text/annotate/MM_2_162_RRB_1863_1980.xml')
text = r.content.decode('utf-8')
text = re.sub(r'(Hirslanden .*? v. Rohstoff)', r'<b>\1</b>', repr(text))
Markdown('<pre>' + text + '</pre>')

<pre>'     Die durch Regierungsbeschluss vom 1.\xa0August l.\xa0Js. für Prüfung der gegen Taxationen der Handelsklassensteuer für das Jahr\xa01863 eingegangenen 179\xa0Rekurse verordnete Commission stellt ihre sachbezüglichen Anträge und verweist zur Begründung derselben auf die von der Finanzdirektion bei den einzelnen Experten, welche bei Festsetzung der Taxationen mitgewirkt haben, eingeholten Gutachten, sowie auf die bei einzelnen Rekurrenten beigesetzten Bemerkungen.    Hierauf hat der Regierungsrath,  nach Einsicht eines Antrages der verordneten Commission,    beschlossen:  I. Werden obige Rekurse in nachstehender Weise erledigt.  II. Wird von der Erledigung der Rekurse denRekurrenten durch die üblichen Formulare, sowie den Statthalterämtern und der Direktion der Finanzen Mittheilung gemacht.       Gemeinde: Name des Rekurrenten: Geschäftszweig: Taxation pro 1863, 1864 Beschluss.    I.\xa0Section:        Frk   Frk.    <b>Hirslanden Hottinger, Konrad Seidenfabrikant 30 30   Wollishofen Asper L. Seidenstofffabrik 40 40   Zürich Andreae, Otto, Commission in Seidenwaaren 300 300.       “ Bäbler, Gebrüder Commiss. Verkauf v. Rohstoff</b> 100 100.       “ Bodmer u. Honegger Commission in Seidenwaaren 350 350       “ Ciolina u. Comp. Modewaaren 150 150       “ Döbeli, J. Commiss. u. Engros Gesch. in Seidenw. 100 70       “ Fierz J.\xa0R. Leinwand u. Baumw. handl. 50 50       “ Geiger, C.\xa0F. Seidenw. u. Commiss. pro rata 88 40       “ Haab, H. Tulle, dentelle, mercer. 80 80       “ Heintze, J. Pelzwaarenhandlung 80 80   [Zürich] Huber u. Geiger Bandfabrikanten 100 50       “ Huber-Zundel (bis er den Austritt aus dem Ragionenbuch erklärt) Seidenstofffbr. in Liquidation 60 40       “ Jenny u. Düring Modewaaren 150 150       “ Knecht, Kaspar Gold- u. Silberwaaren 100 80       “ Meier, J.\xa0J Kleiderhandlung 150 150       “ Mobelmagazin am Eiermarkt von Paur Mobilienhandlung 100 100       “ Nüscheler Christoph Zigarrenfabr. 45 30       “ Ochsner, Joh. Rud. Mobilienhandel 150 150       “ Ochsner-Schaub, F. Mobilienfabrik 40 40    Zürich Passavant, Seemann u. Comp. Commission in Seidenstoffen 600 600       “ Rübel u. Abegg Commission in Seidenwaaren 500 400       “ Scherer, J.\xa0J Tapissier Mobilienhandel 100 100       “ Speerli, Hrch u.\xa0Cie  Modewaarenhandlung 70 50       “ Weber, Heinrich Schuhfabrikation 70 70       “ Wenny\xa0J. Manufakturen 80 80       “ Wichelhausen Joh. Spezerei- u. Wollengarnhdlg. 80 50       “ Willard frères Tuchhandlung en detail 80 80   Hausen Huber S. Tuchhandlung 40 30       “ Zürrer Jakob Seidenfabrikation 400 350   Horgen Guggenheim, Jakob Manufakturwaaren 80 80       “ Stäubli, Julius Seidenfabrikation 80 50   Oberrieden Hotz u. Comp.            “ 100 80          “ Saur, Joh Ulrich Tuch- u. Spezereihandlung 15 15   Wädensweil Müller, Ulrich (bis er den Austritt aus dem Ragionenbuch erklärt) Gold- u. Silberwaaren 15 10           “ Wegmann u. Forster Seidenstofffabrikation 100 80   Küßnacht Staub u. Wegmann Fabrikation u. Seidenstoffen 100 100   Männedorf Dändliker, J.\xa0Jakob Posamenteriewaaren 50 50          “ Zuppinger Joh. (bis er den Austritt aus dem Ragionenbuch erklärt.) Seidenfabrikation 90 20   Stäfa Egli, Johannes Tuchwaarenhandlung 30 30      “ Weinmann, Geschwister Tuch- u. Wollenwaarenhandlg 10 10   Uetikon Leuthold, Jakob Seidenfabrikation 60 60   Fischenthal Keller, J.\xa0J. Broderiewaarenfabrik 15 15   Hinweil Schaufelberger, J\xa0Jb. Quincaillerie, Tuchwaaren etc. 10 10    Mönchaltorf Ringger-Greutert, J.\xa0J. Tuch-[,] Spezerei- u. Quincaillerie 15 15   Pfäffikon Schärer-Gujer Tuchhandlung 30 30   Winterthur Bank Gewähr von Vorschüssen 4000 4000   Zürich Saxer Elise Modewaaren & Parfümerie 40 30   Hirslanden Hardmeier Fr. Seidenfabrikation 50 30.    II. Section:    Riesbach Bader, J. u.\xa0Cie. Seidenzwirnerei 350 300         “ Blatter C. u. Sohn Seidenfärberei 600 600   Unterstraß Ott J.\xa0U. Dezimalwaagenfabr. 25 20   Zürich Hänsler,\xa0H. Seidenfärberei 250 250       “ Wagner-Heim,\xa0Fr. Baumwolle 150 100.       “ Thomann Friedr. Baumwollenhandlung 400 400   Dietikon Boller, J.\xa0Heinrich Baumwollenspinnerei 100 100   Affoltern Salzmann-Gull, J. Seidenfärberei 35 35   Adlisweil Schmied u. Cie  Baumwollspinnerei 250 200   Richtersweil Hürlimann Hrch von Johannes Rothfärberei in Rappersweil 500 400   Wädensweil Baumann u. Frick Baumwollspinnerei 90 80            “ Marthaler Hrch Seidenfärberei 20 10   Uetikon Schmid, Heinrich Baumwollfabrik 20 5   Bärentsweil Spörri, Kaspar Mech. Baumwollweberei 150 100   Fischenthal Keller, Gebrüder Baumwollspinnerei 125 125          “ Müller J.\xa0J. Mech. Baumwollspinnerei 50 40   Hinweil Knecht J. u. Comp Mech. Baumwollweberei 70 50   Rüti Honegger, Heinrich Spinnerei 250 250    Wetzikon Guyer, Flachsmann u. Comp. Kempten Mech. Weberei 150 100        “ Honegger, Rud. Medikon Mech. Werkstätte 150 150   Uster Denzler, Gebrüder Schindelfabrik etc. 30 20      “ Weber, Joß Baumwollfabrikation 25 20   Töß Graf, Louis Mech. Kardenfabrik 20 20   Turbenthal Stahel, Joh. Rud. Fabrikation u. etwas Garnhandel 40 20          “ Winkler, Joh. Rud. z. Friedthal Baumwollspinnerei 500 250   Goßau Heusser, Joh. älter Mechan. Baumwollspinnerei 200 150    III\xa0Section    Altstätten Poirier, Chopat u. fils Chem. Produktenfabrik 300 150   Hirslanden Meyer, Heinr. am Kreuzplatz Spezerei- u. Papierwaaren 45 45   Riesbach Syfrig u. Wieland Lederhandel 30 30   Zürich Boßhard, Hauptagent Hauptagentur d. Helvetia 250 250       “ Büchi, J.\xa0R. Quincaillerie, Kristalhand. 200 200   [Zürich] Egli, Jakob Agenturen u. Commission 80 80       “ Finkbohner, Jean Agenturen 30 30       “ Haas Jos. Sohn Käse- u. Spezereihandel 10 –       “ Hefti G. Spezereihändl. Spezereihandeln 20 15       “ Hermann, A.\xa0W. Münsterhof Mercerie in Commission 20 10       “ Hergt, W.\xa0A. Commission u. Agenturen 100 50       “ Hinder, Louis Mercerie, en gros et en detail 60 60       “ Hitz J\xa0J. Glashandlung 100 70       “ Holzhalb J.\xa0R. Agentur u. Commission 45 30       “ Hüni B. u.\xa0C. (jetzt Singer u. Weber) Fabrik v. Marseillanerseife 35 25    Zürich Käser,\xa0D. Commissionsgeschäft 80 10       “ Koppeler, J. (bis er den Austritt aus dem Ragionenbuch) Spekulationsgeschäft 150 60       “ Kaufmann,\xa0J. Mercerie u. Weißwaaren 100 60       “ Kleinert, Joh. Kolonialwaaren u. Cerealien 70 60       “ Knüsli, Joh. (bis er den Austritt aus dem Ragionenbuch erklärt[)] Agentur 100 60       “ Kuhn, Jak. unterm Hôtel bellevue Spezereihandlung 70 50       “ Löwengard\xa0E. Commissionsgeschäft  80 80       “ Löwenthal,\xa0A. Cigarren 125 100       “ Lohbauer,\xa0C, Sohn älter Papier u. Schreibmaterialien 50 50       “ Louis, Alex. Agentur u. Commission 100  80       “ Reifer J.\xa0H. Colonialwaaren, Glas, Porcelan etc. 80 70       “ Weber u. Aldinger Commission u. Agentur 100 100   Richtersweil Frei Tobias Ellenwaarenhandlung 125 125           “ Nabholz\xa0H. Apotheke 45 45           “ Schneider Heinrich Mehlhandel 40 30   Thalweil Kölliker, Karl Seifen u. Kerzenfabrik 50 50   Wädensweil Staub, Johannes Spezereiwaaren u. Essig 30 30   Uetikon Hauser, J.\xa0R. Fabrik chem. Artikel 20 20   Wald Lier, Rud. im Tobel Spezereihandlung 10 5   Wetzikon Köpf, Mart. Spezerei- u. Mehlhandlung 25 15   Wald Honegger z. Grundstein Spezerei u. Mehlhandlung 10 10   Bauma Wartmann Rudolf Spezerei u. Mehlhandlung 25 20   Pfäffikon Nußbaumer, Felix Spezerei u. Drogeriewaaren 20 15   Töß Bollag, Leopold Ellenwaarenhandlung 30 30    Winterthur Blickle, Moritz Ellenwaaren 125 125          “ Furrer, D. Erhard Gerber Gerberei 60 60          “ Gautschi-Tobler, J. Spezereihandlung 80 80          “ Sulzer,\xa0J. z. Salmen (bis er den Austritt aus dem Ragionenbuch erklärt.[)] Ellenwaaren 35 35          “ Trautwetter\xa0C. z. Tanne Ellenwaaren 50 45          “ Wagner,\xa0R. Seifensiederei 40 20   Niederwenningen Bloch, Moritz Lederhandel 45 20   Zürich Ermatinger\xa0Fl. Agentur für auswärt. Häuser. 80 80       “ Holzhalb,\xa0A. Commission 50 50    IV\xa0Section    Unterstraß Schurter Rudolf Mehl u. Krüschhandel 30 30   Wiedikon Meier, Heinr. z. Meieriesli Pferdehaarfabrik 20 20         “ Mech. Papierfabr. a\xa0d. Sihl Papierfabrikation 1200 1200   Zollikon Leemann-Boller, J\xa0C. Handel m. Ungarnwein 50 30   Zürich Bollinger, Daniel Geschirrhandel 15 15       “ Boßhard Ulrich Eisenhandlung 25 25       “ Hämmerli, Otto Engros Geschäft in Papeterie 20 15       “ Leemann Joh.  Holzhandel 25 10       “ Locher, Kaspar Schwertfegerei pro rata 3 –       “ Pfister, Heinrich Butter u. Weinhandel 40 20       “ Rentenanstalt schweiz. Versicherungsverträge 100 500       “ Schneider, Jean Weinhandlung 100 100       “ Schultheß, Gebrüder Metallgewebe etc. 80 80       “ Stocker Jakob Käsehandel 30 30    Zürich, Sutz, J.\xa0J. Fabrik: v. Sattlerwaaren 40 40       “ Ulrich, J\xa0J. z. Berichthaus Buchdruckerei 350 350   Affoltern Schneebeli. Hs. Hrch. Branntweinbrennerei 10 10   Hausen Weber, Hrch im Hauserthal Branntweinhandel 15 15   Kappel Grob, Jakob im Houen              id          “ Müller, Joh in Uerzlikon              id 15 15        “ Steinmann, Karl              id 15 15        “ Suter, Hs Hrch. in Allenwinden              id 15 15   Mettmenstetten Weiß, Gebrüder im Eigi Schweinhandel 15 15   Wettsweil Baur u. Comp. Ziegelbrennerei 40 40   Hirzel Haab Gebrüder Mühlegewerb 40 30   Horgen Ryf, Conrad auf dem Rohr Sennerei u. Käsehandel 25 10       “ Staub-Streuli Ziegelei 40 30   Richtersweil Hitz, J.\xa0C. Branntweinbrennerei u. Weinhandel 40 \xa050   Schönenberg Hitz, Jakob Branntweinbrennerei 15     Diese beiden Geschäfte sind nun in Eins verschmolzen    Thalweil Kölliker, Hs Jakob Mühlegewerb 15 15        “ Scheller, Kaspar u. Sohn Lampenöhl u. Oehlhandlung 80 70        “ Scheller-Koller u. Comp. Oehlhandlung 80 70   Wädensweil Diener, Jean Ziegelei 50 50           “ Isler, Joh. Säge[-] u. Ladenhandel 30 20           “ Schneider, Kaspar Käse[-] u. Weinhandlung 40 30           “ Höhn, Kaspar Butterhandel 15 15   Hombrechtikon Weber, Jakob Kornhandel 20 20   Meilen Gattiker,\xa0H. im Rosengarten Weinhandlung 80 80    Stäfa Schaufelberger,\xa0D. Weinhandlung 60 60      “ Walder, J.\xa0Jak. Sohn           id 20 15   Feldbach Weber, Charles Mühlegewerb 60 60   Rüti Bünzli, Gebrüder Mühlegewerb u. mechan. Zwirnerei 30 30   Wald Hürlimann, Erhard, Breitenmatt Getränkhandel 10 5   Fehraltorf Bachofner,\xa0Jb Ziegelbrennerei 25 20         “ Schätti, Heinrich Mech. chem. Zündholzfabrik 100 60   Lindau Keller Joh. Rud. in Kleinikon Viehhandel 10 10   Hagenbuch Wettstein,\xa0J. v. Oberschneit        id 15 15   Winterthur Bleuler-Hausheer Buchdruckerei u. Zeitungsverlag 80 50         “ Bruppbacher J.\xa0C. sel. Wittwe Fettwaaren, Wein u. Branntwein 70 50         “ Rieter,\xa0R. Gießerei 200 200         “ Sulzer, Gebrüder      id 1400 1400   Großandelfingen Auer, Conrad Kornhandlung 70 60   Eglisau Frei, Joh. Schmid Eisenwaarenhandlung 10 10        “ Schurter, Melchior Borsten- u. Schweinsdarmhdlg 80 80   Kloten Volkart, Joh. Ladenhandel 15 15   Zürich Weber-Rüsch Waffenhandel 25 25       Der Rekurs des Herrn J\xa0J. Keller, in Fischenthal, Broderiewaaren-Fabrikant wird abgewiesen und dessen Taxation auf 15\xa0frk. belassen.  Ebenso der Rekurs der Bank in Winterthur u. deren Steuer auf Frk.\xa04000 taxirt.    Ferner werden abgewiesen: //   Der Rekurs der Gebrüder Keller in Fischenthal Baumwollspinnerei und die auf 125\xa0Frkn. taxirte Steuer bestätigt,  und der Rekurs des Herrn J.\xa0J.\xa0Ulrich im Berichthaus in Zürich und dessen Steuer auf 350\xa0frkn. belassen.  Dagegen wird der Rekurs der schweiz. Rentenanstalt in Zürich als theilweise begründet gefunden und die auf Frk.\xa01000 angesetzte Steuer auf Fr.\xa0500 ermäßigt.         '</pre>

# Nicht anzeigbares Dokument

## Document-Schnittstelle liefert Dokument

In [38]:
r = requests.get('http://localhost:8080/exist/apps/tei-publisher/api/document/annotate/MM_2_21_RRB_1834_2076.xml')
print(r.content.decode('utf-8'))

<?xml-stylesheet type="text/xsl" href="../../Ressourcen/Stylesheet.xsl"?><TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><teiHeader><fileDesc><titleStmt><title>Der hiesige Stand spricht seine Zustimmung zu Aufstellung eines Bayerischen Consulates in Winterthur aus.</title><respStmt><resp key="transcript" ref="#lmr">Transkript: lmr<date when="2010-02-16">16.02.2010</date></resp><name key="editor">Staatsarchiv des Kantons Zürich</name></respStmt></titleStmt><publicationStmt><authority>Staatsarchiv des Kantons Zürich</authority><date>2010</date></publicationStmt><seriesStmt><title>Regierungsratsbeschlüsse seit 1803 online</title><editor>Staatsarchiv des Kantons Zürich</editor></seriesStmt><sourceDesc><bibl><title>Der hiesige Stand spricht seine Zustimmung zu Aufstellung eines Bayerischen Consulates in Winterthur aus.</title><date when="1834-12-23">23.12.1834</date><ident>StAZH MM 2.21 RRB 1834/2076</ident><idno>83–84</idno><edition>Regierungs

## Plaintext kann jedoch nicht extrahiert werden

In [39]:
r = requests.get('http://localhost:8080/exist/apps/tei-publisher/api/nlp/text/annotate/MM_2_21_RRB_1834_2076.xml')
text = r.content.decode('utf-8')
print(repr(text))

''


## Im TEI-Publisher weder in der entsprechenden Collection aufgeführt, noch direkt aufrufbar

http://localhost:8080/exist/apps/tei-publisher/annotate/MM_2_21_RRB_1834_2076.xml?view=div&odd=annotations

Im TEI-PUblisher (eingeloggt mit Benutzer, der Lese- und Schreibrechte hat):

> Die Anfrage an den Server schlug fehl.: Document annotate/MM_2_21_RRB_1834_2076.xml not found [at line 537 column 9 in module /db/apps/tei-publisher/modules/lib/api/document.xql]

In der Shell:

```
19 Feb 2024 19:17:15,023 [qtp1191863711-187] ERROR (LogFunction.java [writeLog]:199) - (Line: 521 /exist/etc/../data/expathrepo/roaster-1.8.1/content/router.xql) [d699be75-c2cb-424a-99be-75c2cb724a57] get /login: 404
            {"code":"errors:NOT_FOUND_404","description":"document login not found","value":null,"module":"/db/apps/tei-publisher/modules/lib/api/view.xql","line":45,"column":9} 
19 Feb 2024 19:17:15,077 [qtp1191863711-171] ERROR (LogFunction.java [writeLog]:199) - (Line: 521 /exist/etc/../data/expathrepo/roaster-1.8.1/content/router.xql) [85ca205f-ff40-4883-8a20-5fff40d88370] get /api/parts/annotate%2FMM_2_21_RRB_1834_2076.xml/json: 404
            {"code":"errors:NOT_FOUND_404","description":"Document annotate/MM_2_21_RRB_1834_2076.xml not found","value":null,"module":"/db/apps/tei-publisher/modules/lib/api/document.xql","line":537,"column":9} 
19 Feb 2024 19:17:15,093 [qtp1191863711-112] ERROR (LogFunction.java [writeLog]:199) - (Line: 521 /exist/etc/../data/expathrepo/roaster-1.8.1/content/router.xql) [5d9b762a-f4f8-4076-9b76-2af4f89076b2] get /api/parts/annotate%2FMM_2_21_RRB_1834_2076.xml/json: 404
            {"code":"errors:NOT_FOUND_404","description":"Document annotate/MM_2_21_RRB_1834_2076.xml not found","value":null,"module":"/db/apps/tei-publisher/modules/lib/api/document.xql","line":537,"column":9
```

## Versus Dokument, das tatsächlich nicht existiert:

Im TEI-Publisher:
 
> An error has occurred
> 
> An error has been generated by the application:
> 
> document annotate/does_not_exist.xml not found


In der Shell (keine Fehler seitens document.xql):

```
19 Feb 2024 19:19:38,085 [qtp1191863711-192] ERROR (LogFunction.java [writeLog]:199) - (Line: 521 /exist/etc/../data/expathrepo/roaster-1.8.1/content/router.xql) [91625ff4-fecb-4a65-a25f-f4fecbfa65c8] get /annotate/does_not_exist.xml: 404
            {"code":"errors:NOT_FOUND_404","description":"document annotate/does_not_exist.xml not found","value":null,"module":"/db/apps/tei-publisher/modules/lib/api/view.xql","line":45,"column":9} 
19 Feb 2024 19:19:38,232 [qtp1191863711-192] ERROR (LogFunction.java [writeLog]:199) - (Line: 521 /exist/etc/../data/expathrepo/roaster-1.8.1/content/router.xql) [061c0e61-241e-496c-9c0e-61241e096c15] get /login: 404
            {"code":"errors:NOT_FOUND_404","description":"document login not found","value":null,"module":"/db/apps/tei-publisher/modules/lib/api/view.xql","line":45,"column":9}
```